In [1]:
import mdtraj as md
import numpy as np
import os

In [2]:
# inputdir, outputdir
inputdir = "input"

outputdir = "output"
os.makedirs(outputdir, exist_ok=True)

In [3]:
trj = md.load_trr(f"{inputdir}/out.trr", top=f"{inputdir}/solv_ions_prot.gro")
print(trj)

<mdtraj.Trajectory with 500 frames, 4792 atoms, 309 residues, and unitcells>


In [4]:
trj.xyz.shape

(500, 4792, 3)

# make weight list

In [5]:
from weightdict import atomicWeightsDecimal as wdict
weight_key = 'standard'

atoms_dict = trj.topology.to_dataframe()[0].element

wlist = [float(wdict[atom][weight_key]) for atom in atoms_dict]

# centering

In [6]:
# before centering
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

146172.1851362678
146932.03135434075
148223.58173471055
147453.68239108718
146938.58363073212
146701.03182547682
145840.4915579726
144539.3844968831
144239.59204599247
145374.07823754655


In [7]:
# centering
trj.center_coordinates(mass_weighted=True)

<mdtraj.Trajectory with 500 frames, 4792 atoms, 309 residues, and unitcells at 0x1c335814828>

In [8]:
# after centering
for t in range(10):
    tmpxyz = trj.xyz[t]

    total_x = 0
    for i in range(tmpxyz.shape[0]):
        total_x = total_x + tmpxyz[i][0]*wlist[i]
    
    print(total_x)

-0.0020182300616653492
-0.0023475031443247474
-0.0022486422465028966
-0.0015107285236837242
-0.0015292740585834963
-0.0015892604226621643
-0.0028540748965362184
-0.002422328964321707
-0.0018043812778678614
-0.0026877619365563987


# choose 2 frames

In [9]:
a = trj.xyz[0]
b = trj.xyz[-1]

# matrix U

In [10]:
U = np.empty((3,3))

for i in range(3):
    for j in range(3):
        U[i][j] = sum( [wlist[n]*a[n,i]*b[n,j] for n in range(a.shape[0])] )

print(U)

[[ 62952.17991029  38570.72318614 -33253.59856558]
 [ -7879.77936183  37143.47651281  -5878.48978125]
 [ 13068.04244232   8980.31518004  14457.95218838]]


# 固有値問題

In [11]:
# 6*6のomega定義
OMEGA = np.empty((6,6))

for i in range(3):
    for j in range(3):
        OMEGA[i][j] = 0
        OMEGA[i+3][j+3] = 0
        OMEGA[i+3][j] = U[i][j]
        OMEGA[i][j+3] = U.T[i][j]

print(OMEGA)

[[     0.              0.              0.          62952.17991029
   -7879.77936183  13068.04244232]
 [     0.              0.              0.          38570.72318614
   37143.47651281   8980.31518004]
 [     0.              0.              0.         -33253.59856558
   -5878.48978125  14457.95218838]
 [ 62952.17991029  38570.72318614 -33253.59856558      0.
       0.              0.        ]
 [ -7879.77936183  37143.47651281  -5878.48978125      0.
       0.              0.        ]
 [ 13068.04244232   8980.31518004  14457.95218838      0.
       0.              0.        ]]


In [12]:
# 固有値問題
eig_val, eig_vec =np.linalg.eig(OMEGA)

omegas = eig_vec.T

print("pairs of eig")
for i in range(len(eig_val)):
    print('----------')
    print(eig_val[i])
    print(omegas[i])

pairs of eig
----------
82912.57364980305
[ 0.52024659  0.39369797 -0.27266352  0.68750629  0.14625927  0.07709301]
----------
-82912.573649803
[ 0.52024659  0.39369797 -0.27266352 -0.68750629 -0.14625927 -0.07709301]
----------
35148.677261257166
[ 4.18700695e-01 -5.69343625e-01 -2.31854403e-02  1.47064578e-01
 -6.91644101e-01  6.68567349e-04]
----------
19577.8616175314
[ 0.23245047  0.1443944   0.65201    -0.07554547 -0.01538383  0.70289133]
----------
-35148.67726125715
[-4.18700695e-01  5.69343625e-01  2.31854403e-02  1.47064578e-01
 -6.91644101e-01  6.68567349e-04]
----------
-19577.86161753139
[-0.23245047 -0.1443944  -0.65201    -0.07554547 -0.01538383  0.70289133]


In [13]:
hks = np.array([omegas[i] for i in range(6) if eig_val[i] > 0])
hks = hks * np.sqrt(2) # root2を出す

In [14]:
k = np.empty((3,3))
h = np.empty((3,3))
for i in range(3):
    for j in range(3):
        k[i][j] = hks[j][i]
        h[i][j] = hks[j][i+3]

# 回転行列R

In [15]:
R = np.empty((3,3))

for i in range(3):
    for j in range(3):
        R[i][j] = sum( [k[i][a]*h[j][a] for a in range(len(h))] )
print(R)

[[ 0.80337652 -0.43415391  0.40754945]
 [ 0.35206241  0.8982876   0.26292858]
 [-0.48024809 -0.06774781  0.87451244]]


In [16]:
# 正規直交確認
print(R@R.T)

[[ 1.00000000e+00  4.82446326e-16 -8.04668102e-16]
 [ 4.82446326e-16  1.00000000e+00  8.01157461e-16]
 [-8.04668102e-16  8.01157461e-16  1.00000000e+00]]


# bをrへ変換

In [17]:
r = np.empty_like(b)

for i in range(3):
    for n in range(r.shape[0]):
        r[n,i] = sum( [R[i,j]*b[n,j] for j in range(3)] )

# save

In [18]:
fitted_xyz = np.array([a,r])

fitted_xyz.shape

(2, 4792, 3)

In [19]:
fitted_trj = md.Trajectory(fitted_xyz, trj.topology)

In [20]:
fitted_trj.save_trr(f"{outputdir}/test.trr")

In [21]:
type(a)

numpy.ndarray